# Desafio - Enviando um Email

### Vocês precisam fazer o seguinte:

* A partir da função, criem uma tool
* Desenvolvam o argschema da tool, lembrando que todos os argumentos são strings.
* Combinem a tool a um modelo de linguagem.
* Enviem um email utilizando o modelo de linguagem para ‘aluno@asimov.academy’ com o título ‘Enviando um email com llm’ e uma mensagem bonita no corpo do texto.

In [6]:
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

In [14]:
from langchain.agents import tool
from email.message import EmailMessage
from pydantic import BaseModel, Field
import smtplib
import ssl

class EnviaEmailArgs(BaseModel):
    destinatario: str = Field(description='Destinatário do E-mail')
    titulo: str = Field(description='Título do E-mail, deve ser breve e intuitivo')
    corpo: str = Field(description='Corpo do E-mail. Deve ser breve, formal e abordando o assunto com clareza')

@tool(args_schema=EnviaEmailArgs)
def envia_email(destinatario: str, titulo: str, corpo: str):
    """Envia emails para um destinatário com um título e corpo"""
    email_usuario = 'marissaborges2006@gmail.com'
    senha_app = 'edzt hxgo rtvt wtsy'
    message_email = EmailMessage()
    message_email['From'] = email_usuario
    message_email['To'] = destinatario
    message_email['Subject'] = titulo

    message_email.set_content(corpo)
    safe = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=safe) as smtp:
        smtp.login(email_usuario, senha_app)
        smtp.sendmail(email_usuario, destinatario, message_email.as_string())

In [21]:
from langchain.prompts import ChatPromptTemplate as cpt
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai.chat_models import ChatOpenAI

prompt = cpt.from_messages([
    ('system', 'Você é um assistente amigável e formal de envio de e-mails'),
    ('user', '{input}')
])

chat = ChatOpenAI(model='gpt-4o-mini')

tools = [envia_email]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chat_com_func = chat.bind(functions=tools_json)

In [22]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [23]:
chain = prompt | chat_com_func | OpenAIFunctionsAgentOutputParser() | roteamento

In [26]:
chain.invoke({'input': 'Envie um e-mail para aluno@asimov.academy com o título "Enviando um email com llm" e uma mensagem bonita no corpo do texto dizendo o quanto essa escola de python é incrível'})